# Dataset Preprocessing For token classification
## for Encouragement Approach (Discourage avoidance) only

In [ ]:
import pandas as pd
import numpy as np
import os
from datasets import Dataset, DatasetDict
from datasets import Dataset, DatasetDict, ClassLabel, Value

# 1. All data without any filter

In [ ]:
path_dataset = '/home/lossioventuraj2/Code/07-Transcripts/words_w_exp_fine_V1/'
subsets = ['2_training', '3_validation', '4_testing']

In [ ]:
list_dataframes = []
csv_per_set = {}

list_length_tokens = []

for subset_ in subsets:
    name_dataset = subset_[2:]
    dataframes = {}
    print(name_dataset)
    list_files = os.listdir(path_dataset + subset_)
    print(f"\tNumber of files: {len(list_files)}")
    for file_ in list_files:
        df_aux = pd.read_csv(path_dataset + subset_ + "/" + file_)
        dataframes[file_[:-4]] = df_aux
        list_length_tokens.append(df_aux.shape[0])
        list_dataframes.append(df_aux)
        #print(df_aux.shape)

    csv_per_set[name_dataset] = dataframes

In [ ]:
features = {
    'id': Value('string'),
    'tokens': Value('string'),
    'labels': ClassLabel(names=["NoDA","DA"])  
}

In [ ]:
print(len(csv_per_set['training']))

In [ ]:
list_for_Datasets = {}
MAX_LENGTH = 256

for set_, dict_of_dataframes in csv_per_set.items():
    print(set_)
    print(len(dict_of_dataframes))

    dataset_list = []
    for name, df in dict_of_dataframes.items():
        tokens = df['WORD'].tolist()
        labels = df['DiscourageAvoidance'].tolist()
    
        #print(len(tokens))
        for i in range(0, len(tokens), MAX_LENGTH):
            #print(i)
            dict_aux_ = {
                'id': str(name) + f"_part{i // MAX_LENGTH + 1}",  # Convert ID to string
                'tokens': [str(token) for token in tokens[i:i + MAX_LENGTH]],  # Make sure string type
                'ner_tags': [int(label) for label in labels[i:i + MAX_LENGTH]]   # Make sure string type
            }
            dataset_list.append(dict_aux_)

    dataset_aux = Dataset.from_list(dataset_list)
    list_for_Datasets[set_] = dataset_aux

dataset_dict = DatasetDict(list_for_Datasets)

In [ ]:
print(type(dataset_dict))  

# Save Dataset

In [ ]:
dataset_dict.save_to_disk("discourage_avoidance_all")

In [ ]:
training_dataset = dataset_dict["training"]
print(type(training_dataset))  

In [ ]:
dataset_dict

# Load Dataset

In [ ]:
from datasets import DatasetDict

# Load the DatasetDict from the saved directory
loaded_dataset_dict = DatasetDict.load_from_disk("discourage_avoidance_all")

# Now `loaded_dataset_dict` is the same as the original dataset_dict
print(loaded_dataset_dict)

# 2. Changing sequence lower than 10 to 0
# Since it might add noise instead.

In [ ]:
THRESHOLD = 10

In [ ]:
path_dataset = '/home/lossioventuraj2/Code/07-Transcripts/words_w_exp_fine_V1/'
subsets = ['2_training', '3_validation', '4_testing']

In [ ]:
def modify_short_sequences(df):
    df = df.copy()  # Copy to avoid SettingWithCopyWarning
    in_sequence = False
    start_idx = None

    for i in range(len(df)):
        if df.loc[i, "DiscourageAvoidance"] == 1:
            if not in_sequence:
                in_sequence = True
                start_idx = i
        else:
            if in_sequence:
                in_sequence = False
                end_idx = i
                length = end_idx - start_idx

                if length < THRESHOLD:
                    df.loc[start_idx:end_idx, "DiscourageAvoidance"] = 0  # Make sure modification
    
    if in_sequence and (len(df) - start_idx) < THRESHOLD:
        df.loc[start_idx:, "DiscourageAvoidance"] = 0

    return df 

In [ ]:
list_dataframes = []
csv_per_set = {}

list_length_tokens = []

for subset_ in subsets:
    name_dataset = subset_[2:]
    dataframes = {}
    print(name_dataset)
    list_files = os.listdir(path_dataset + subset_)
    print(f"\tNumber of files: {len(list_files)}")
    for file_ in list_files:
        df_aux = pd.read_csv(path_dataset + subset_ + "/" + file_)
        dataframes[file_[:-4]] = df_aux
        list_length_tokens.append(df_aux.shape[0])
        list_dataframes.append(df_aux)

    csv_per_set[name_dataset] = dataframes

In [ ]:
list_for_Datasets = {}
MAX_LENGTH = 256

for set_, dict_of_dataframes in csv_per_set.items():
    print(set_)
    print(len(dict_of_dataframes))

    dataset_list = []
    for name, df in dict_of_dataframes.items():

        print("Before processing:\n")
        print(df["DiscourageAvoidance"].value_counts())
        df = modify_short_sequences(df)
        print("\nAfter processing:\n")
        print(df["DiscourageAvoidance"].value_counts())
        print("\n####################################\n")
        
        tokens = df['WORD'].tolist()
        labels = df['DiscourageAvoidance'].tolist()
    
        #print(len(tokens))
        for i in range(0, len(tokens), MAX_LENGTH):
            #print(i)
            dict_aux_ = {
                'id': str(name) + f"_part{i // MAX_LENGTH + 1}",  # Convert ID to string
                'tokens': [str(token) for token in tokens[i:i + MAX_LENGTH]],  # Make Sure string type
                'ner_tags': [int(label) for label in labels[i:i + MAX_LENGTH]]   # Make Sure string type
            }
            dataset_list.append(dict_aux_)

    dataset_aux = Dataset.from_list(dataset_list)
    list_for_Datasets[set_] = dataset_aux

dataset_dict_filtered = DatasetDict(list_for_Datasets)

In [ ]:
print(type(dataset_dict_filtered))  

In [ ]:
dataset_dict_filtered.save_to_disk("discourage_avoidance_10")

# 3. Merge close sequences
## If the gap (number of 0s) between a sequence end and the next sequence start 
## is less than 10, convert those 0s into 1s. 

In [ ]:
THRESHOLD = 10

In [ ]:
def merge_close_sequences(df):
    df = df.copy()
    in_sequence = False
    sequences = []  # Store (start, end) of sequences

    # Identify sequences of 1s
    for i in range(len(df)):
        if df.loc[i, "DiscourageAvoidance"] == 1:
            if not in_sequence:
                start_idx = i
                in_sequence = True
        else:
            if in_sequence:
                end_idx = i - 1  # Last 1 in the sequence
                sequences.append((start_idx, end_idx))
                in_sequence = False

    # Handle case if the sequence extends to the last row
    if in_sequence:
        sequences.append((start_idx, len(df) - 1))

    # If no sequences were found, return the DataFrame unchanged
    if not sequences:
        return df

    # Merge sequences that are close together
    merged_sequences = []
    prev_start, prev_end = sequences[0]

    for i in range(1, len(sequences)):
        curr_start, curr_end = sequences[i]
        
        if (curr_start - prev_end) < 10:  # If the gap is less than 10
            prev_end = curr_end  # Merge the sequence
        else:
            merged_sequences.append((prev_start, prev_end))
            prev_start, prev_end = curr_start, curr_end
    
    merged_sequences.append((prev_start, prev_end))  # Add the last sequence

    # Preserve existing 1s and update only necessary 0s
    for start, end in merged_sequences:
        df.loc[start:end, "DiscourageAvoidance"] = 1  # Make sure all merged areas are 1

    return df


In [ ]:
list_dataframes = []
csv_per_set = {}

list_length_tokens = []

for subset_ in subsets:
    name_dataset = subset_[2:]
    dataframes = {}
    print(name_dataset)
    list_files = os.listdir(path_dataset + subset_)
    print(f"\tNumber of files: {len(list_files)}")
    for file_ in list_files:
        df_aux = pd.read_csv(path_dataset + subset_ + "/" + file_)
        dataframes[file_[:-4]] = df_aux
        list_length_tokens.append(df_aux.shape[0])
        list_dataframes.append(df_aux)
        #print(df_aux.shape)

    csv_per_set[name_dataset] = dataframes

In [ ]:
list_for_Datasets = {}
MAX_LENGTH = 256

for set_, dict_of_dataframes in csv_per_set.items():
    print(set_)
    print(len(dict_of_dataframes))

    dataset_list = []
    for name, df in dict_of_dataframes.items():
        df = df.copy()
        print("Before processing:")
        print(df["DiscourageAvoidance"].value_counts())
        df = merge_close_sequences(df)
        print("After processing:")
        print(df["DiscourageAvoidance"].value_counts())
        
        tokens = df['WORD'].tolist()
        labels = df['DiscourageAvoidance'].tolist()
    
        #print(len(tokens))
        for i in range(0, len(tokens), MAX_LENGTH):
            #print(i)
            dict_aux_ = {
                'id': str(name) + f"_part{i // MAX_LENGTH + 1}",  # Convert ID to string
                'tokens': [str(token) for token in tokens[i:i + MAX_LENGTH]],  # Make sure string type
                'ner_tags': [int(label) for label in labels[i:i + MAX_LENGTH]]   # Make sure string type
            }
            dataset_list.append(dict_aux_)

    dataset_aux = Dataset.from_list(dataset_list)
    list_for_Datasets[set_] = dataset_aux

dataset_dict_filled = DatasetDict(list_for_Datasets)

In [ ]:
"ID": str(name_file),
"Exposure_Number": str(number_exposure),
"ID_ExNum": str(name), 
"Sub_ID": int(i+1),
"Indexes": [int(indx_) for indx_ in range(i,i + window_size)],
"tokens": [str(token) for token in window],  #window,
"ner_tags": [str(cat) for cat in window_label] #window_label

In [ ]:
list_for_Datasets = {}
MAX_LENGTH = 250

for set_, dict_of_dataframes in csv_per_set.items():
    print(set_)
    print(len(dict_of_dataframes))

    dataset_list = []
    for name, df in dict_of_dataframes.items():
        
        name_file = name[:-17]
        number_exposure = name[-4:]
        
        df = df.copy()
        print("Before processing:")
        print(df["DiscourageAvoidance"].value_counts())
        df = merge_close_sequences(df)
        print("After processing:")
        print(df["DiscourageAvoidance"].value_counts())
        
        tokens = df['WORD'].tolist()
        labels = df['DiscourageAvoidance'].tolist()
    
        #print(len(tokens))
        for i in range(0, len(tokens), MAX_LENGTH):
            #print(i)
            dict_aux_ = {
                "ID": str(name_file),
                "Exposure_Number": str(number_exposure),
                "ID_ExNum": str(name), 
                "Sub_ID": int(i // MAX_LENGTH + 1),
                'tokens': [str(token) for token in tokens[i:i + MAX_LENGTH]],  # Make sure string type
                'ner_tags': [int(label) for label in labels[i:i + MAX_LENGTH]]   # Make sure string type
            }
            dataset_list.append(dict_aux_)

    dataset_aux = Dataset.from_list(dataset_list)
    list_for_Datasets[set_] = dataset_aux

dataset_dict_filled = DatasetDict(list_for_Datasets)

In [ ]:
dataset_dict_filled.save_to_disk("datasets/no_slide_windows/window_250/discourage_avoidance_merged_alt")

In [ ]:
dataset_dict_filled